# Build an Agent

## Define tools

In [1]:
pip install --quiet langchain-anthropic langgraph


[notice] A new release of pip is available: 23.0.1 -> 24.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
from langchain_community.tools.tavily_search import TavilySearchResults

search = TavilySearchResults(max_results=2)
search_results = search.invoke("what is the weather in SF")
print(search_results)
# If we want, we can create other tools.
# Once we have all the tools we want, we can put them in a list that we will reference later.
tools = [search]

[{'url': 'https://www.weatherapi.com/', 'content': "{'location': {'name': 'San Francisco', 'region': 'California', 'country': 'United States of America', 'lat': 37.78, 'lon': -122.42, 'tz_id': 'America/Los_Angeles', 'localtime_epoch': 1719414277, 'localtime': '2024-06-26 8:04'}, 'current': {'last_updated_epoch': 1719414000, 'last_updated': '2024-06-26 08:00', 'temp_c': 13.3, 'temp_f': 55.9, 'is_day': 1, 'condition': {'text': 'Overcast', 'icon': '//cdn.weatherapi.com/weather/64x64/day/122.png', 'code': 1009}, 'wind_mph': 10.5, 'wind_kph': 16.9, 'wind_degree': 280, 'wind_dir': 'W', 'pressure_mb': 1017.0, 'pressure_in': 30.04, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 93, 'cloud': 100, 'feelslike_c': 12.5, 'feelslike_f': 54.5, 'windchill_c': 12.2, 'windchill_f': 53.9, 'heatindex_c': 13.1, 'heatindex_f': 55.6, 'dewpoint_c': 10.4, 'dewpoint_f': 50.6, 'vis_km': 16.0, 'vis_miles': 9.0, 'uv': 4.0, 'gust_mph': 15.0, 'gust_kph': 24.1}}"}, {'url': 'https://www.wunderground.com/hourly/us/ca/

In [4]:
search_results = search.invoke("what is the weather in SF")
print(search_results)
tools = [search]

[{'url': 'https://www.wunderground.com/hourly/us/ca/san-francisco/94117/date/2024-6-26', 'content': 'Current Weather for Popular Cities . San Francisco, CA 62 ° F Partly Cloudy; Manhattan, NY 82 ° F Fair; Schiller Park, IL (60176) 81 ° F Cloudy; Boston, MA 81 ° F Partly Cloudy; Houston, TX ...'}, {'url': 'https://www.weatherapi.com/', 'content': "{'location': {'name': 'San Francisco', 'region': 'California', 'country': 'United States of America', 'lat': 37.78, 'lon': -122.42, 'tz_id': 'America/Los_Angeles', 'localtime_epoch': 1719414277, 'localtime': '2024-06-26 8:04'}, 'current': {'last_updated_epoch': 1719414000, 'last_updated': '2024-06-26 08:00', 'temp_c': 13.3, 'temp_f': 55.9, 'is_day': 1, 'condition': {'text': 'Overcast', 'icon': '//cdn.weatherapi.com/weather/64x64/day/122.png', 'code': 1009}, 'wind_mph': 10.5, 'wind_kph': 16.9, 'wind_degree': 280, 'wind_dir': 'W', 'pressure_mb': 1017.0, 'pressure_in': 30.04, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 93, 'cloud': 100, 'feel

## Using Language Models

In [5]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4")

In [6]:
model_with_tools = model.bind_tools(tools)

In [8]:
from langchain_core.messages import HumanMessage
response = model_with_tools.invoke([HumanMessage(content="Hi!")])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

ContentString: Hello! How can I assist you today?
ToolCalls: []


In [9]:
response = model_with_tools.invoke([HumanMessage(content="What's the weather in SF?")])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

ContentString: 
ToolCalls: [{'name': 'tavily_search_results_json', 'args': {'query': 'current weather in San Francisco'}, 'id': 'call_GOzfWH9dFvQxY0t264FFq4ee'}]


## Create the agent

In [10]:
from langgraph.prebuilt import create_react_agent

agent_executor = create_react_agent(model, tools)

## Run the agent

In [11]:
response = agent_executor.invoke({"messages": [HumanMessage(content="hi!")]})

response["messages"]

[HumanMessage(content='hi!', id='cf3b3e95-2844-4ee2-bd9f-6fe9989fc88e'),
 AIMessage(content='Hello! How can I assist you today?', response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 83, 'total_tokens': 93}, 'model_name': 'gpt-4-0613', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-ce61dac5-2706-4b10-880e-ad0b156f2f6b-0', usage_metadata={'input_tokens': 83, 'output_tokens': 10, 'total_tokens': 93})]